# Read Dataframe

Reads "property_and_listings_data.csv" into a dataframe with proper formatting & data types. 

In [141]:
import pandas as pd
import numpy as np
import plotly.express as px
import re

DTYPES = {
    'property_id': np.uint32, 
    'property_URL': 'str', 
    'address': 'str',
    'apartment_number': 'str', 
    'object_type': 'str', 
    'latitude': np.float64, 
    'longitude': np.float64,
    'construction_year': np.float64, # IN REALITY INT; BUT EASIER THIS WAY
    'energy_class': 'str', 
    #'descriptive_area_name': 'str',
    'has_solar_panels': 'str', 
    'brf_name': 'str', 
    'brf_url': 'str', 
    'montly_payment': 'str', # TO BE CHANGED 
    'rent': np.float64, # IN REALITY INT; BUT EASIER THIS WAY
    'rooms': np.float64, # MEMORY COULD BE GREATLY REDUCED 
    'sqm': np.float64, # MEMORY COULD BE GREATLY REDUCED 
    'primary_area': 'str', 
    'floor': 'str', 
    'operating_cost': np.float64, # IN REALITY INT; BUT EASIER THIS WAY
    #'estimate_price' # Parsed to int using converters later
    #'estimate_low'  # Parsed to int using converters later
    #'estimate_high'  # Parsed to int using converters later
    'listing_agent': 'str',
    'listing_agency_name': 'str', 
    'listing_agency_URL': 'str', 
    'listing_days_active': np.uint32,
    'listing_sold_date': 'str', 
    'listing_sold_price_type': 'str', 
    #'listing_sold_price'  # Parsed to int using converters later
    #'listing_listed_price' # Parsed to int using converters later
}

def parse_price_string(x):
    price = re.sub(" (kr)?", "", x)
    if price != "":
        return int(price)
    else:
        return None

# Change price to be integer instead of formatted string
CONVERTERS = {
    c : lambda x : parse_price_string(x) for c in [
        "estimate_price",
        "estimate_low",
        "estimate_high",
        "listing_sold_price",
        "listing_listed_price"
    ]
}

# Align area names format somewhat
CONVERTERS["descriptive_area_name"] = lambda x : re.sub(" ?(-|/) ?", " ", x).title()

DATA_DIR = r"C:\Users\asus\Desktop\Workspace\Booli_scraper\property_data\property_and_listings_data.csv"

df = pd.read_csv(DATA_DIR, 
                 sep=";", 
                 encoding="utf8", 
                 dtype=DTYPES, 
                 converters=CONVERTERS,
                 parse_dates=["listing_sold_date"], 
                 index_col=0, 
                 low_memory=False)

# Density heatmap plot

Try to visualize data using density heatmap. Not ideal though for visualizing price, since the density means areas with many listings will outweigh areas with high prices etc. Could be useful for other purposes in the future.

In [102]:
recent_sales = df[df["listing_sold_date"].apply(lambda x : x.year) > 2020]

# Add a price per sqm column
recent_sales = recent_sales.assign(
    listing_sold_price_per_sqm=recent_sales["listing_sold_price"] / recent_sales["sqm"]
)

#recent_sales = recent_sales.groupby("primary_area")["latitude", "longitude", "listing_sold_price_per_sqm"].mean()

# Heatmap
fig = px.density_mapbox(
    recent_sales, lat="latitude", 
    lon="longitude", 
    z="listing_sold_price_per_sqm",
    center=dict(lat=recent_sales["latitude"].mean(), lon=recent_sales["longitude"].mean()), 
    zoom=10,
    mapbox_style="stamen-terrain",
    hover_name=recent_sales.index,
    radius=6
)

fig.show()

# Scatter plot

A better way to visualize price data etc. However, improved area groupings would be beneficial to get fine grained insights without dilating the quality.

In [142]:
recent_sales = df[df["listing_sold_date"].apply(lambda x : x.year) > 2015]

# Add a price per sqm column
recent_sales = recent_sales.assign(
    listing_sold_price_per_sqm=recent_sales["listing_sold_price"] / recent_sales["sqm"]
)

# Filter out areas with too few listings
areas = recent_sales["descriptive_area_name"].value_counts()[recent_sales["descriptive_area_name"].value_counts()>10].index
recent_sales = recent_sales[recent_sales["descriptive_area_name"].isin(areas)]

recent_sales = recent_sales.groupby("descriptive_area_name")["latitude", "longitude", "listing_sold_price_per_sqm"].mean()

fig = px.scatter_mapbox(
    recent_sales, 
    lat="latitude", 
    lon="longitude",     
    color="listing_sold_price_per_sqm", 
    size="listing_sold_price_per_sqm",
    #color_continuous_scale=px.colors.cyclical.IceFire, 
    #size_max=15, 
    hover_name=recent_sales.index,
    zoom=10
)
fig.update_layout(mapbox_style="carto-positron")
fig.show()

c:\users\asus\desktop\workspace\booli_scraper\venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

